Installing the required packages

In [ ]:
import torch

# 1. Manually install the missing dependency from Git first
print("--- Fixing Dependency Mismatch (Installing unsloth-zoo) ---")
!pip install --upgrade "git+https://github.com/unslothai/unsloth-zoo.git"

# 2. Now install Unsloth (It should find the dependency now)
print("\n--- Installing Unsloth ---")
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 3. Re-verify dependencies (Just to be safe)
print("\n--- Verifying Dependencies ---")
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

print("\n✅ DONE. Please RESTART your session now (Runtime > Restart Session).")

--- Fixing Dependency Mismatch (Installing unsloth-zoo) ---
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-req-build-10a8_55b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-req-build-10a8_55b
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit ba585aff2b3f80594497171c8d6d216f921cdb8d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 958.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 44.4 MB/s eta 0:00:00
   

Loading the Model (Qwen3B)

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None

# --- CONFIGURATION: QLoRA vs LoRA ---
# Set True for QLoRA (Uses ~2.5GB VRAM, faster loading)
# Set False for Standard LoRA (Uses ~6.5GB VRAM, slightly higher precision)
load_in_4bit = False  # <--- CHANGED TO FALSE FOR STANDARD LoRA

model, tokenizer = FastLanguageModel.from_pretrained(
    # We remove the "-bnb-4bit" suffix so it finds the full model to load in 16-bit
    model_name = "unsloth/Qwen2.5-Coder-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # Passed False here
)

print(f"SUCCESS: Qwen 3B loaded! (4-bit Quantization: {load_in_4bit})")

==((====))==  Unsloth 2026.2.1: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

SUCCESS: Qwen 3B loaded! (4-bit Quantization: False)


Loading and formatting the training data

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
import os

# 1. Load the dataset
dataset_path = "/content/unsloth_training_data.json"
print(f"✅ Loading {dataset_path}...")
dataset = load_dataset("json", data_files=dataset_path, split="train")

# 2. Apply QWEN-2.5 Chat Formatting
# This is the critical step that teaches the model to understand "user" vs "assistant"
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",  # <--- Specifically for Qwen
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return { "text" : texts }

# 3. Apply the formatting
dataset = dataset.map(formatting_prompts_func, batched = True)

# 4. Verify the output
print("SUCCESS: Data loaded and formatted for Qwen!")
print(f"\nSample Input (Check for <|im_start|> tags):\n{dataset['text'][0][:300]}...")

✅ Loading /content/unsloth_training_data.json...


Map:   0%|          | 0/4043 [00:00<?, ? examples/s]

SUCCESS: Data loaded and formatted for Qwen!

Sample Input (Check for <|im_start|> tags):
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Schema: {
  "db_id": "department_management",
  "collection_names": [
    "head",
    "management",
    "department"
  ],
  "column_names": [
    [
      0,
      "_id"
    ],
    [
   ...


In [ ]:
import os

# 1. Increase the timeout limit (Default is 10s, we set to 120s)
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "120"

# 2. (Optional) Enable faster downloads if available
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

print("✅ Network timeout increased. You can now try loading the model again.")

✅ Network timeout increased. You can now try loading the model again.


Fine-Tuning and saving the Model to the drive

In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from transformers.trainer_utils import get_last_checkpoint
from google.colab import drive
import os

# --- 1. SETUP: Mount Drive & Define Checkpoint Path ---
drive.mount('/content/drive')
# Checkpoints will be saved here. Change the name "QwenTraining_Run1" if you start a new experiment.
drive_output_dir = "/content/drive/MyDrive/QwenTraining_Run1"

print(f"📂 Checkpoints will be saved to: {drive_output_dir}")

# --- 2. Load Model in 4-bit ---
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-3B-Instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# --- 3. Add Adapters ---
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)

# --- 4. Load & Format Data ---
dataset_path = "/content/unsloth_training_data.json"

# Safety Check: If runtime crashed, /content/ files are wiped. Remind user to re-upload.
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"❌ DATA MISSING: {dataset_path} not found.\n"
                            "Since the runtime likely crashed, please Re-Upload 'unsloth_training_data.json' to the Colab Files tab!")

dataset = load_dataset("json", data_files=dataset_path, split="train")
tokenizer = get_chat_template(tokenizer, chat_template = "qwen-2.5")

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return { "text" : texts }

dataset = dataset.map(formatting_prompts_func, batched = True)

# --- 5. Training Configuration (With Drive Saving) ---
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,

    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        gradient_checkpointing = True,

        # --- Epoch Based Training ---
        num_train_epochs = 1,
        max_steps = -1,

        # --- Optimizer ---
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        warmup_ratio = 0.03,

        # --- CRITICAL: Save to Google Drive ---
        output_dir = drive_output_dir,     # Save directly to Drive
        save_strategy = "steps",
        save_steps = 50,                   # Save every 50 steps
        save_total_limit = 2,              # Keep only last 2 checkpoints (saves space)
        logging_steps = 1,
        seed = 3407,
        report_to = "none",
    ),
)

# --- 6. Intelligent Auto-Resume Logic ---
print("-" * 50)
last_checkpoint = get_last_checkpoint(drive_output_dir)

if last_checkpoint:
    print(f"🔄 FOUND CHECKPOINT in Drive: {last_checkpoint}")
    print("🚀 RESUMING training from where we left off...")
    trainer_stats = trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("🆕 No existing checkpoints found.")
    print("🚀 Starting FRESH training...")
    trainer_stats = trainer.train()

print("✅ Training Complete!")

Mounted at /content/drive
📂 Checkpoints will be saved to: /content/drive/MyDrive/QwenTraining_Run1
==((====))==  Unsloth 2026.2.1: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/4043 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/4043 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


--------------------------------------------------
🆕 No existing checkpoints found.
🚀 Starting FRESH training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,043 | Num Epochs = 1 | Total steps = 253
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 119,734,272 of 3,205,672,960 (3.74% trained)


Step,Training Loss
1,0.997300
2,1.096800
3,0.946500
4,1.031000
5,0.888200
6,0.724700
7,0.688700
8,0.579900
9,0.630500
10,0.637300


✅ Training Complete!
